In [ ]:
import pandas as pd
import pickle
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
import concurrent.futures
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
from sklearn.cluster import OPTICS
from sklearn.metrics import silhouette_score
from sklearn.model_selection import ParameterSampler
from sklearn.base import BaseEstimator
from sklearn.model_selection import LeaveOneOut, KFold


In [ ]:
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)  

In [ ]:
df = pd.read_parquet('COMPRESSED_CIS_tle_data.parquet')
df.shape

In [ ]:
df.info()

**Describe the fields:**

0. idElset: A unique identifier for each satellite element set. Object data type.

1. classificationMarking: Indicates the classification of the satellite data, where "U" stands for unclassified. Object data type.

2. satNo: The satellite number, a unique identifier for a satellite. Interger data type.

3. epoch: The timestamp indicating when the satellite's position and orbital elements were recorded. This is in UTC format. Currently as object data type.

4. meanMotion: The number of revolutions the satellite completes around the earth per day. This parameter gives an idea of the speed at which the satellite orbits the earth. 64-bit floating-point number.

5. idOnOrbit: A repeat of the satellite number (satNo), possibly for consistency across different data sources. Object data type.

6. eccentricity: A measure of how much the satellite's orbit deviates from a perfect circle. Values closer to 0 indicate a more circular orbit, while values closer to 1 indicate a more elliptical orbit. 64-bit floating-point number.

7. inclination: The angle between the satellite's orbital plane and the Earth's equator, measured in degrees. It indicates how far north or south the satellite travels relative to the equator. 64-bit floating-point number.

8. raan (Right Ascension of the Ascending Node): The angle, measured from a fixed point in the sky, representing where the satellite crosses the equator from the southern hemisphere to the northern hemisphere. 64-bit floating-point number.

9. argOfPerigee (Argument of Perigee): The angle within the satellite's orbital plane that defines the point where the satellite is closest to the Earth (perigee). 64-bit floating-point number.

10. meanAnomaly: An angular parameter that helps describe the position of the satellite along its orbit at a specific time. 64-bit floating-point number.

11. revNo: The revolution number of the satellite, indicating the number of complete orbits made by the satellite since launch. An interger data type.

12. bStar: A drag term used in the satellite's orbital model to account for atmospheric drag affecting the satellite’s motion, particularly in lower orbits. 64-bit floating-point number.

13. meanMotionDot: The first derivative of the mean motion, representing how the satellite’s speed is changing over time, which might be due to perturbing forces like gravitational effects. 64-bit floating-point number.

14. meanMotionDDot: The second derivative of the mean motion, further describing changes in the satellite’s speed, providing more detailed information about the satellite's acceleration or deceleration. 64-bit floating-point number.

15. semiMajorAxis: The distance from the center of the satellite's orbit to its farthest point, measured in kilometers. This value is important for describing the size of the satellite's orbit. 64-bit floating-point number.

16. period: The time it takes for the satellite to complete one full orbit around the Earth, measured in minutes. 64-bit floating-point number.

17. apogee: The highest point in the satellite's orbit, i.e., the farthest distance from the Earth during its orbit. 64-bit floating-point number.

18. perigee: The lowest point in the satellite's orbit, i.e., the closest distance to the Earth during its orbit. 64-bit floating-point number.

19. line1: A string representing the first line of a Two-Line Element (TLE) format, which contains information about the satellite's orbit. Object data type.

20. line2: A string representing the second line of a TLE format, which includes additional details about the satellite's orbital elements. Object data type.

21. createdAt: The timestamp indicating when this specific record or data entry was created. Object data type.

22. createdBy: The identifier for the system or person responsible for generating this specific record. Object data type.

23. source: The organization or system that provided the satellite data. In this case, "18th SPCS" refers to the 18th Space Control Squadron, which tracks space objects. Object data type.

24. dataMode: Describes the type or mode of data being used or collected. For example, 'REAL' might indicate real-time or current data. Object data type.

25. origNetwork: Likely identifies the network or system through which the data was originally collected or processed. Object data type.

26. algorithm: The algorithm used to process or analyze the satellite data. In this case, 'SGP4' refers to the Simplified General Perturbations model, which is a commonly used orbital model for predicting satellite positions. Object data type.

27. sourceDL: Possibly refers to additional details or metadata about the data source, though it’s marked as None in the provided example. Object data type.

28. ephemType: Likely refers to the type of ephemeris (data that provides the positions of astronomical objects), though it is marked as NaN in this example. 64-bit floating-point number. 

29. uct: May refer to Universal Coordinated Time (UTC), though this column is marked as None in the example provided. Object data type.

30. descriptor: Additional descriptive information about the satellite or data, though it is marked as None in this case. Object data type.

In [ ]:
# Load the CIS_satcat.pkl dataset
file_path = r'C:\Users\megan\OneDrive\Desktop\Data_Analytics\CIS 831A\Project\CIS831_russat-main\output\CIS_satcat.pkl'

with open(file_path, 'rb') as f:
    satcat = pickle.load(f)

# Convert the satcat data into a DataFrame for easy manipulation
satcat_df = pd.DataFrame(satcat)

# Ensure 'NORAD_CAT_ID' and 'satNo' are the same type, converting both to integers
satcat_df['NORAD_CAT_ID'] = satcat_df['NORAD_CAT_ID'].astype(int)
df['satNo'] = df['satNo'].astype(int)

# Merge the original dataset with the new data on 'satNo' and 'NORAD_CAT_ID'
# Keep only the 'COUNTRY' and 'OBJECT_TYPE' field from the new data
enriched_df = pd.merge(df, satcat_df[['NORAD_CAT_ID', 'COUNTRY', 'OBJECT_TYPE']], 
                       how='inner', left_on='satNo', right_on='NORAD_CAT_ID')

# Filter to include only 'CIS' country and 'ROCKET BODY' or 'PAYLOAD'
filtered_df = enriched_df[(enriched_df['COUNTRY'] == 'CIS') & 
                          (enriched_df['OBJECT_TYPE'].isin(['PAYLOAD']))]

# Apply the orbital features filter to select only the relevant columns
orbital_features = [
    'epoch', 'meanMotion', 'eccentricity', 'inclination', 
    'raan', 'argOfPerigee', 'meanAnomaly', 'meanMotionDot', 
    'meanMotionDDot', 'semiMajorAxis', 'apogee', 'perigee'
]

# Add the necessary columns for identification (e.g., 'satNo', 'COUNTRY', 'OBJECT_TYPE')
necessary_columns = ['satNo', 'COUNTRY', 'OBJECT_TYPE'] + orbital_features

# Filter the DataFrame to keep only the necessary columns
filtered_df = filtered_df[necessary_columns]

# Drop Duplications
filtered_df = filtered_df.drop_duplicates()

# Display the filtered data to ensure it worked
filtered_df.head()

In [ ]:
filtered_df.info()

**Method:**

This example is designed to detect and visualize anomalies in satellite orbital data, focus on rates of change for two key orbital features: Semi-Major Axis and Inclination. It identifies significant deviations in these features as potential indicators of satellite maneuvers, then visualizes and counts anomalies over time. Dure the previous step, I limited the data to just Russian Payload data.

1. Data Preprocessing: Satellite data, grouped by satellite ID, is preprocessed by converting the time feature (epoch) to UTC and calculating the rate of change for the orbital features: Semi-Major Axis and Inclination.

2. Anomaly Detection: Thresholds are set for each feature to detect significant changes, which may indicate intentional satellite movement. These thresholds are 10 for the Semi-Major Axis and 0.1 for Inclination. Points exceeding these thresholds are classified as anomalies.

3. Normalization: To standardize the values for Semi-Major Axis and Inclination across different satellites, the data is normalized using StandardScaler to allow for fair comparisons of orbital changes across different satellites.

4. Anomaly Visualization: For each satellite, the anomalies are visualized in a time series plot where significant deviations are highlighted. This helps in understanding if the detected changes are potentially intentional.

5. Interactive Aggregation: Anomalies are aggregated by day and visualized interactively using Plotly, allowing zooming capabilities for more focused analysis of specific time ranges.

In [ ]:
# Function to calculate the rate of change for a given feature
def calculate_rate_of_change(sat_group, feature, time_column='epoch'):
    delta_feature = sat_group[feature].diff().abs()
    delta_time = sat_group[time_column].diff().dt.total_seconds() / (3600 * 24)  # convert time difference to days
    rate_of_change = delta_feature / delta_time
    return rate_of_change

# Set thresholds for significant changes in Semi-Major Axis and Inclination
semi_major_axis_threshold = 10  # Adjust this based on your dataset (e.g., large maneuvers)
inclination_threshold = 0.1      # Example threshold for inclination change

# Group the data by satellite
grouped = filtered_df.groupby('satNo')

# List to store anomalies
all_anomalies = []

# Create a scaler to normalize the values
scaler = StandardScaler()

for sat_no, sat_group in grouped:
    print(f"Processing satellite {sat_no}")
    
    # Convert 'epoch' to datetime if not already done
    sat_group['epoch'] = pd.to_datetime(sat_group['epoch'], errors='coerce')
    
    # Drop rows with invalid 'epoch' values (if any)
    sat_group = sat_group.dropna(subset=['epoch'])
    
    # Sort by 'epoch' to maintain time order
    sat_group = sat_group.sort_values('epoch')

    # Ensure the 'epoch' column is in UTC (if not already)
    if sat_group['epoch'].dt.tz is None:
        sat_group['epoch'] = sat_group['epoch'].dt.tz_localize('UTC')

    # Calculate rate of change for Semi-Major Axis and Inclination
    sat_group['semi_major_axis_rate_of_change'] = calculate_rate_of_change(sat_group, 'semiMajorAxis')
    sat_group['inclination_rate_of_change'] = calculate_rate_of_change(sat_group, 'inclination')

    # Normalize both Semi-Major Axis and Inclination for standardized scale
    sat_group[['semiMajorAxis', 'inclination']] = scaler.fit_transform(sat_group[['semiMajorAxis', 'inclination']])

    # Identify points with significant changes
    anomalies = sat_group[(sat_group['semi_major_axis_rate_of_change'] > semi_major_axis_threshold) |
                          (sat_group['inclination_rate_of_change'] > inclination_threshold)]

    if not anomalies.empty:
        # Store anomalies for this satellite for the final summary plot
        all_anomalies.append(anomalies)

        # Get OBJECT_TYPE for the current satellite
        object_type = sat_group['OBJECT_TYPE'].iloc[0]  # Assuming 'OBJECT_TYPE' is consistent per satellite

        # Visualize the anomalies for this satellite with OBJECT_TYPE in the title
        plt.figure(figsize=(12, 6))
        plt.plot(sat_group['epoch'], sat_group['semiMajorAxis'], label='Semi-Major Axis', color='blue')
        plt.plot(sat_group['epoch'], sat_group['inclination'], label='Inclination', color='orange')
        plt.scatter(anomalies['epoch'], anomalies['semiMajorAxis'], color='red', label='Anomalous Semi-Major Axis', marker='x')
        plt.scatter(anomalies['epoch'], anomalies['inclination'], color='green', label='Anomalous Inclination', marker='o')
        plt.xlabel('Epoch (UTC)')
        plt.ylabel('Standardized Orbital Parameters')
        plt.title(f"Anomalies for Satellite {sat_no} ({object_type}) - Potential Maneuvers")
        plt.legend()
        plt.show()

# Step 3: Combine all anomalies
combined_anomalies = pd.concat(all_anomalies)

# Step 4: Separate anomalies by OBJECT_TYPE
rocket_bodies = combined_anomalies[combined_anomalies['OBJECT_TYPE'] == 'ROCKET BODY']
payloads = combined_anomalies[combined_anomalies['OBJECT_TYPE'] == 'PAYLOAD']

# Count the number of anomalies per epoch for each type
anomalies_per_epoch_rocket_bodies = rocket_bodies.groupby('epoch').size().reset_index(name='anomaly_count')
anomalies_per_epoch_payloads = payloads.groupby('epoch').size().reset_index(name='anomaly_count')

# Step 5: Create Plotly figure to plot the count of anomalies over time (epoch)
# Group anomalies by day
combined_anomalies['epoch_day'] = combined_anomalies['epoch'].dt.floor('D')  # Group by day
anomalies_per_day = combined_anomalies.groupby('epoch_day').size().reset_index(name='anomaly_count')

# Create a zoomable plot using Plotly
fig = px.line(anomalies_per_day, x='epoch_day', y='anomaly_count', title='Anomaly Count Over Time (Grouped by Day)',
              labels={'epoch_day': 'Date (UTC)', 'anomaly_count': 'Anomaly Count'},
              markers=True)

# Add zoom capability
fig.update_xaxes(rangeslider_visible=True)

# Show the plot
fig.show()

# Print the total number of anomalies detected
print(f"Total anomalies detected: {combined_anomalies.shape[0]}")

## OPTICS (Ordering Points To Identify the Clustering Structure)

Density-based Clustering: OPTICS is particularly good at identifying clusters of varying densities, which is useful in scenarios where some satellite movements might follow tight, controlled patterns while others might have larger deviations (possibly intentional).

Hierarchical Clustering: Unlike DBSCAN, OPTICS allows for the creation of hierarchical clusters, which can help in identifying outliers that don't belong to any cluster and track shifts in movement patterns more accurately.

No Need to Specify Cluster Count: Similar to DBSCAN, you don’t need to pre-define the number of clusters. This can be useful when analyzing satellite data, where the number of clusters (potential movement patterns or anomalies) isn’t always known.

Detection of Both Anomalous Clusters and Points: OPTICS provides a reachability plot that can highlight outliers that don't belong to any particular cluster, making it ideal for detecting satellites with potential intentional movement.

**Methodology:**

1. Custom Silhouette Scorer for OPTICS: The code begins by defining a custom scoring function, optics_silhouette_scorer. This function applies the Silhouette Score to measure the quality of clusters generated by OPTICS, a clustering algorithm. If there is only one cluster (i.e., no meaningful clustering), the function returns a low score (-1), ensuring that the model will prefer configurations that produce more than one cluster.

2. Hyperparameter Tuning: The RandomizedSearchCV method is used to perform hyperparameter tuning for the OPTICS model. It randomly samples from predefined hyperparameter values (min_samples and max_eps), which control the minimum number of samples for a core point and the maximum distance between points in the same cluster, respectively. The tuning process evaluates 10 combinations of these parameters, and the custom silhouette scorer helps identify the best combination of hyperparameters that produces well-separated clusters.

3. Satellite Data Preprocessing: Each satellite group is processed separately. Data preprocessing steps include:
    - Datetime conversion: The 'epoch' column is converted to a datetime format.
    - Sorting and timezone localization: The data is sorted by time, and the 'epoch' is localized to UTC if needed.
    - Imputation of missing values: Missing values in the key features (semiMajorAxis and inclination) are filled using the mean imputation strategy.
    - Normalization: The orbital parameters are scaled to a standardized range using StandardScaler.

4. Rate of Change Calculation: The rate of change for key orbital parameters (semi-major axis and inclination) is calculated to capture significant variations over time, which might indicate satellite maneuvers.

5. OPTICS Anomaly Detection: For each satellite, after preprocessing, the OPTICS algorithm is applied to detect clusters and outliers. The best configuration of hyperparameters (found using RandomizedSearchCV) is used to fit the data. The OPTICS labels are extracted, with outliers (labeled as -1) indicating anomalous points in the satellite's trajectory.

6. Anomaly Visualization: For each satellite with detected anomalies, the orbital features (semi-major axis and inclination) are plotted over time, and anomalies are highlighted with different markers on the plots. Each plot is titled with the satellite number and object type.

7. Summary and Final Plot: The anomalies detected across all satellites are combined, and a final Plotly visualization is created. The plot shows the anomaly count over time, grouped by day, to provide a time-based trend of detected satellite anomalies.

In [ ]:
# Define a custom scoring function for OPTICS using silhouette score
def optics_silhouette_scorer(estimator, X):
    labels = estimator.fit_predict(X)
    if len(set(labels)) > 1:
        return silhouette_score(X, labels)
    else:
        return -1

# Define the hyperparameter space for OPTICS
param_distributions = {
    'min_samples': [5, 10, 25, 50],
    'max_eps': [0.5, 1.0, 1.5, 2.0]
}

# Process each satellite
for sat_no, sat_group in grouped:
    print(f"Processing satellite {sat_no}")
    
    sat_group['epoch'] = pd.to_datetime(sat_group['epoch'], errors='coerce')
    sat_group = sat_group.dropna(subset=['epoch']).sort_values('epoch')
    
    if sat_group['epoch'].dt.tz is None:
        sat_group['epoch'] = sat_group['epoch'].dt.tz_localize('UTC')

    # Imputation and scaling
    sat_group[['semiMajorAxis', 'inclination']] = imputer.fit_transform(sat_group[['semiMajorAxis', 'inclination']])
    sat_group[['semiMajorAxis', 'inclination']] = scaler.fit_transform(sat_group[['semiMajorAxis', 'inclination']])

    # Select features for clustering
    features_to_cluster = sat_group[['semiMajorAxis', 'inclination']].values

    if len(features_to_cluster) > 1:
        # For satellites with more than 1 sample, use KFold or RandomizedSearchCV
        optics_model = OPTICS()
        
        # Adjust cross-validation based on the number of samples
        if len(features_to_cluster) >= 5:
            cv = KFold(n_splits=5)
        else:
            cv = LeaveOneOut()  # For small datasets
        
        random_search = RandomizedSearchCV(
            estimator=optics_model,
            param_distributions=param_distributions,
            n_iter=10,
            random_state=42,
            scoring=optics_silhouette_scorer,
            cv=cv
        )
        
        random_search.fit(features_to_cluster)
        best_model = random_search.best_estimator_
    else:
        print(f"Skipping satellite {sat_no} due to insufficient data.")
        continue
    
    # Apply the best model
    best_model.fit(features_to_cluster)
    sat_group['optics_label'] = best_model.labels_
    
    # Identify anomalies
    anomalies = sat_group[sat_group['optics_label'] == -1]
    
    if not anomalies.empty:
        all_anomalies.append(anomalies)

        object_type = sat_group['OBJECT_TYPE'].iloc[0]
        plt.figure(figsize=(12, 6))
        plt.plot(sat_group['epoch'], sat_group['semiMajorAxis'], label='Semi-Major Axis', color='blue')
        plt.plot(sat_group['epoch'], sat_group['inclination'], label='Inclination', color='orange')
        plt.scatter(anomalies['epoch'], anomalies['semiMajorAxis'], color='red', marker='x', label='Anomalous Semi-Major Axis')
        plt.scatter(anomalies['epoch'], anomalies['inclination'], color='green', marker='o', label='Anomalous Inclination')
        plt.xlabel('Epoch (UTC)')
        plt.ylabel('Standardized Orbital Parameters')
        plt.title(f"Anomalies for Satellite {sat_no} ({object_type}) - Potential Maneuvers")
        plt.legend()
        plt.show()
#Removed consolidated plot for overall activity, rather do something different following this.


Next Actions to add...

0. Explain adding another feature such as mean_motion
1. Label the anomalies back on the orginal data set
2. Plot consolidated activity windows based on anomalies detected
3. Plot over map physical location the during the anomaly windows.

## Paired T-Test Comparison